**テーマ**　：　迷路の最短ルートを導き出したい

**基礎となるアルゴリズム**　：　迷路作成、報酬の設定、Q学習を用いた強化学習、最短ルートの表示

**実行プログラム、その結果**　：　下記




**今後自分の研究を深める上で必要なツール**　：　道の最短経路を用いるためQ学習を使うと考えられる。


**Numpy、Pandas、Matplotlibといった講義で説明したモジュールを利用していること**　：　Numpyを使用


**何らかのデータを入力、出力すること**　：　迷路のデータ、迷路の報酬の設定、最短経路を作成し出力している

# **Q学習**
[参考先](https://note.com/pumonmon/n/n04f9139ad826)

In [ ]:
def get_action(state, action, observation, reward):
   next_state = digitize_state(observation)
   next_action = np.argmax(q_table[next_state])

   alpha = 0.2
   gamma = 0.99
   q_table[state, action] = (1 - alpha) * q_table[state, action] +\
           alpha * (reward + gamma * q_table[next_state, next_action])
   return next_action, next_state
for episode in range(num_episodes):

   observation = env.reset()
   state = digitize_state(observation)
   action = np.argmax(q_table[state])
   episode_reward = 0
   for t in range(max_number_of_steps):

       env.render()

       observation, reward, done, info = env.step(action)

       action, state = get_action(state, action, observation, reward)
       episode_reward += reward

# **強化学習の報酬の基準**

進む先が正しいときは報酬の１を渡す。

進先＜ゴールとなるようにゴールの報酬を大きくする。

迷路を大きくしたいが手作業でrewardを作るのはめんどくさいのでまずはその部分のプログラムを作る。

追記:ランダムで迷路を作成するものも欲しくなったため、それも作成。順番は前後した。

# **迷路を作る**

In [83]:
import numpy as np
import random

# 迷路の大きさ
maze_size = [5,5]

# スタートとゴールの位置
start = (0, 0)
goal =list(map(lambda x: x - 1, maze_size))

def make_maze(maze_size, start, goal):
    # 迷路を全て壁（0）で初期化
    maze = np.zeros(maze_size, dtype=int)

    # スタート地点を1にする
    maze[start] = 1

    # 方向ベクトル（上下左右）
    directions = [(2, 0), (-2, 0), (0, 2), (0, -2)]

    def is_valid_move(x, y):
        # 迷路の範囲内かチェック
        return 0 <= x < maze_size[0] and 0 <= y < maze_size[1]

    def carve_passage_from(x, y):
        # 方向をランダムにシャッフル
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(nx, ny) and maze[nx, ny] == 0 :
                # 壁を壊して道を作る
                maze[nx, ny] = 1
                maze[x + dx // 2, y + dy // 2] = 1
                carve_passage_from(nx, ny)

    # スタート地点から穴を掘る
    carve_passage_from(*start)

    # ゴール地点を道にする
    maze[goal] = 1

    return maze

maze = make_maze(maze_size, start, goal)
print(maze)
[0, 1, 2, 7, 12, 17, 22, 23, 24]

[[1 1 1 0 1]
 [0 0 1 0 1]
 [1 1 1 0 1]
 [1 0 1 0 1]
 [1 1 1 1 1]]


# **迷路を報酬つきにする**

In [84]:
import numpy as np

# 迷路の定義（スタートは0、ゴールは8）
# 1は通れる道、0は壁
#maze_sizeの迷路を0,1で作成

"""
迷路作成時に設定済

# 迷路の大きさ
maze_size = (5, 5)

# スタートとゴールの位置
start = (0, 0)
goal = (2, 2)

#スタートとゴールの位置を1にする
maze[start] = 1
maze[goal] = 1

#mazeが機能しているかの確認
print(maze)
"""

#迷路のreward化(通れるかどうかを判定)
def make_reward(maze):
    size = maze.shape[0] * maze.shape[1]
    reward = np.zeros((size, size), dtype = int)

    #上下左右に1があれば1を置く
    for i in range(maze.shape[0]):
        for j in range(maze.shape[1]):
            if maze[i, j] == 1:
                current_state = i * maze.shape[1] + j
                if i > 0 and maze[i-1, j] == 1:  # 上に移動可能
                    next_state = (i-1) * maze.shape[1] + j
                    reward[current_state, next_state] = 1
                if i < maze.shape[0] - 1 and maze[i+1, j] == 1:  # 下に移動可能
                    next_state = (i+1) * maze.shape[1] + j
                    reward[current_state, next_state] = 1
                if j > 0 and maze[i, j-1] == 1:  # 左に移動可能
                    next_state = i * maze.shape[1] + (j-1)
                    reward[current_state, next_state] = 1
                if j < maze.shape[1] - 1 and maze[i, j+1] == 1:  # 右に移動可能
                    next_state = i * maze.shape[1] + (j+1)
                    reward[current_state, next_state] = 1


    #ゴールの報酬部分の入れ替え
    for i in range(reward.shape[0]):
      if reward[i, (reward.shape[0] - 1)] == 1:
        reward[i, (reward.shape[0] - 1)] = 100000

    return reward


# reward行列の生成
reward = make_reward(maze)

#rewardの確認
print(reward)


[[     0      1      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     1      0      1      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      1      0      0      0      0      0      1      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      1
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     0      0      0      0      0      0      0      0      0      0
       0      0   

# **最終テスト時**

In [85]:
import numpy as np
gamma = 0.9 #小さいと行動直後の利益を重視。
alpha = 0.1 #学習率
num = 10000 #何回行うか

#Q値(行動価値)の初期値を設定
Q = np.zeros((maze_size[0] * maze_size[1], maze_size[0] * maze_size[1]))

#Q学習を実装し、各位置における行動価値を算出
for i in range(num):#numの回数繰り返し学習を行う
    p_state = np.random.randint(0, maze_size[0] * maze_size[1])#現在の状態をランダムに選択
    n_actions = []#次の行動の候補
    for j in range(maze_size[0] * maze_size[1]):
        if reward[p_state,j] >= 1:
            n_actions.append(j) #p_stateの状態で移動できる場所を取得
    if len(n_actions) > 0:
        n_state = np.random.choice(n_actions)
        Q[p_state, n_state] = (1 - alpha) * Q[p_state, n_state] + alpha * (reward[p_state, n_state] + gamma * Q[n_state, np.argmax(Q[n_state, :])])
    else:
        # p_stateから移動可能なアクションがない場合
        continue

    #Q値の更新
    Q[p_state,n_state] = (1-alpha)*Q[p_state,n_state]+alpha*(reward[p_state,n_state]+gamma*Q[n_state,np.argmax(Q[n_state,])])
#最短ルート表示関数の定義。Q値が最も高い行動をappendで追加していく
def shortest_path(start):#startに依存する
    path = [start]#pathに経路を追加していく
    p_pos = start#p_posは現在位置
    n_pos = p_pos#n_posにいったんp_posを代入
    while(n_pos != (maze_size[0]**2 - 1)):#n_posがゴールになるまで繰り返し行動を選択
        n_pos = np.argmax(Q[p_pos,])#各位置の行動価値が最も高い行動を選択
        path.append(n_pos)#経路をpathに追加
        p_pos = n_pos#行動後が次のp_posとなる
    return path
print(shortest_path(0))#スタートを0として最短経路を表示

[0, 1, 2, 7, 12, 17, 22, 23, 24]
